<a href="https://colab.research.google.com/github/niharsheth24/Hello-World-/blob/master/Stock_Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import CustomBusinessDay
import os
import glob
import warnings
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")
os.system('rm /content/bhavcopies/eq*')
def EOD(start,end):
    bday=CustomBusinessDay(holidays=['2021-01-26','2021-03-11','2021-03-29','2021-04-02','2021-04-14','2021-04-21','2021-05-13','2021-07-21','2021-08-19','2021-09-10','2021-10-15','2021-11-05','2021-11-19'])
    dt = pd.date_range(start,end,freq=bday)
    for tday in dt:  
                    dd=datetime.strftime(tday,"%d")
                    mm=datetime.strftime(tday,"%m")
                    yyyy=datetime.strftime(tday,"%Y")
                    url = f'https://archives.nseindia.com/products/content/sec_bhavdata_full_'+dd+mm+yyyy+'.csv'
                    #print(url)
                    datafutures=pd.read_csv(url)
                    #print(datafutures)
                    data=datafutures[~datafutures.duplicated(subset=['SYMBOL'])]
                    #print(data)
                    data.to_csv(r'/content/bhavcopies/eq'+dd+mm+'.csv',index=False)
                    
EOD('13/06/2021', '23/06/2021')

#COMBINE ALL FILES IN EQ

os.chdir(r'/content/bhavcopies')
extension = 'csv' 
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv_eq = pd.concat([pd.read_csv(f) for f in all_filenames ])
#combined_csv_eq.to_csv( "Eq.csv", index=False, encoding='utf-8-sig')

In [2]:
combined_csv_eq.head()

,SYMBOL,SERIES,DATE1,PREV_CLOSE,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,LAST_PRICE,CLOSE_PRICE,AVG_PRICE,TTL_TRD_QNTY,TURNOVER_LACS,NO_OF_TRADES,DELIV_QTY,DELIV_PER
0,20MICRONS,EQ,22-Jun-2021,63.55,64.50,65.15,62.55,62.90,62.95,63.52,156829,99.61,2599,82823,52.81
1,21STCENMGM,EQ,22-Jun-2021,19.85,19.90,20.20,19.90,20.20,20.20,20.07,2367,0.48,13,2367,100.00
2,3IINFOTECH,BE,22-Jun-2021,11.00,11.55,11.55,10.55,10.85,10.90,11.36,45004127,5114.21,41120,-,-
3,3MINDIA,EQ,22-Jun-2021,25411.90,25302.00,25649.00,25197.35,25598.00,25572.35,25425.78,1610,409.35,1030,723,44.91
4,3PLAND,EQ,22-Jun-2021,13.80,14.10,14.10,13.75,13.95,13.80,13.86,16898,2.34,119,13683,80.97


In [3]:
df = combined_csv_eq[combined_csv_eq[' SERIES']==' EQ']

In [4]:
dates = df[' DATE1'].unique()
dates = dates.tolist()
dates.sort(reverse=True)

In [5]:
dates

[' 23-Jun-2021',
 ' 22-Jun-2021',
 ' 21-Jun-2021',
 ' 18-Jun-2021',
 ' 17-Jun-2021',
 ' 16-Jun-2021',
 ' 15-Jun-2021',
 ' 14-Jun-2021']

In [6]:
d1 = df[df[' DATE1']==dates[0]]
d2 = df[df[' DATE1']==dates[1]]
d3 = df[df[' DATE1']==dates[-1]]

In [7]:
merged = pd.merge(d1,d2,on=['SYMBOL'],suffixes=['_tday', '_prev'])
merged1 = pd.merge(d1,d3,on=['SYMBOL'],suffixes=['_tday', '_prev'])

In [8]:
merged.head()

,SYMBOL,SERIES_tday,DATE1_tday,PREV_CLOSE_tday,OPEN_PRICE_tday,HIGH_PRICE_tday,LOW_PRICE_tday,LAST_PRICE_tday,CLOSE_PRICE_tday,AVG_PRICE_tday,TTL_TRD_QNTY_tday,TURNOVER_LACS_tday,NO_OF_TRADES_tday,DELIV_QTY_tday,DELIV_PER_tday,SERIES_prev,DATE1_prev,PREV_CLOSE_prev,OPEN_PRICE_prev,HIGH_PRICE_prev,LOW_PRICE_prev,LAST_PRICE_prev,CLOSE_PRICE_prev,AVG_PRICE_prev,TTL_TRD_QNTY_prev,TURNOVER_LACS_prev,NO_OF_TRADES_prev,DELIV_QTY_prev,DELIV_PER_prev
0,20MICRONS,EQ,23-Jun-2021,62.95,63.80,64.2,61.80,62.15,62.05,62.74,121773,76.40,2051,76978,63.21,EQ,22-Jun-2021,63.55,64.5,65.15,62.55,62.90,62.95,63.52,156829,99.61,2599,82823,52.81
1,21STCENMGM,EQ,23-Jun-2021,20.20,20.60,20.6,20.60,20.60,20.60,20.60,1405,0.29,10,1405,100.00,EQ,22-Jun-2021,19.85,19.9,20.20,19.90,20.20,20.20,20.07,2367,0.48,13,2367,100.00
2,3MINDIA,EQ,23-Jun-2021,25572.35,25403.05,25725.5,25336.00,25464.00,25495.55,25571.29,1329,339.84,948,596,44.85,EQ,22-Jun-2021,25411.90,25302.0,25649.00,25197.35,25598.00,25572.35,25425.78,1610,409.35,1030,723,44.91
3,3PLAND,EQ,23-Jun-2021,13.80,13.85,14.0,13.75,13.80,13.90,13.82,21933,3.03,89,20431,93.15,EQ,22-Jun-2021,13.80,14.1,14.10,13.75,13.95,13.80,13.86,16898,2.34,119,13683,80.97
4,5PAISA,EQ,23-Jun-2021,421.70,425.70,426.6,416.00,418.00,416.75,419.23,19921,83.51,934,11999,60.23,EQ,22-Jun-2021,415.80,422.6,431.35,419.30,421.40,421.70,423.51,28102,119.01,2226,13090,46.58


In [9]:
merged[' DELIV_QTY_tday'] = merged[' DELIV_QTY_tday'].astype(int)
merged[' DELIV_QTY_prev'] = merged[' DELIV_QTY_prev'].astype(int)
merged[' AVG_PRICE_tday'] = merged[' AVG_PRICE_tday'].astype(int)

merged1[' DELIV_QTY_tday'] = merged1[' DELIV_QTY_tday'].astype(int)
merged1[' DELIV_QTY_prev'] = merged1[' DELIV_QTY_prev'].astype(int)
merged1[' AVG_PRICE_tday'] = merged1[' AVG_PRICE_tday'].astype(int)

In [10]:
merged['%CHG'] = (merged[' CLOSE_PRICE_tday'] - merged[' PREV_CLOSE_tday']) / merged[' PREV_CLOSE_tday'] * 100
merged['%CHG_DELIV_QTY'] = (merged[' DELIV_QTY_tday'] / merged[' DELIV_QTY_prev'])

merged1['%CHG'] = (merged1[' CLOSE_PRICE_tday'] - merged1[' PREV_CLOSE_tday']) / merged1[' PREV_CLOSE_tday'] * 100
merged1['%CHG_DELIV_QTY'] = (merged1[' DELIV_QTY_tday'] / merged1[' DELIV_QTY_prev'])

In [11]:
merged

,SYMBOL,SERIES_tday,DATE1_tday,PREV_CLOSE_tday,OPEN_PRICE_tday,HIGH_PRICE_tday,LOW_PRICE_tday,LAST_PRICE_tday,CLOSE_PRICE_tday,AVG_PRICE_tday,TTL_TRD_QNTY_tday,TURNOVER_LACS_tday,NO_OF_TRADES_tday,DELIV_QTY_tday,DELIV_PER_tday,SERIES_prev,DATE1_prev,PREV_CLOSE_prev,OPEN_PRICE_prev,HIGH_PRICE_prev,LOW_PRICE_prev,LAST_PRICE_prev,CLOSE_PRICE_prev,AVG_PRICE_prev,TTL_TRD_QNTY_prev,TURNOVER_LACS_prev,NO_OF_TRADES_prev,DELIV_QTY_prev,DELIV_PER_prev,%CHG,%CHG_DELIV_QTY
0,20MICRONS,EQ,23-Jun-2021,62.95,63.80,64.2,61.80,62.15,62.05,62,121773,76.40,2051,76978,63.21,EQ,22-Jun-2021,63.55,64.50,65.15,62.55,62.90,62.95,63.52,156829,99.61,2599,82823,52.81,-1.429706,0.929428
1,21STCENMGM,EQ,23-Jun-2021,20.20,20.60,20.6,20.60,20.60,20.60,20,1405,0.29,10,1405,100.00,EQ,22-Jun-2021,19.85,19.90,20.20,19.90,20.20,20.20,20.07,2367,0.48,13,2367,100.00,1.980198,0.593578
2,3MINDIA,EQ,23-Jun-2021,25572.35,25403.05,25725.5,25336.00,25464.00,25495.55,25571,1329,339.84,948,596,44.85,EQ,22-Jun-2021,25411.90,25302.00,25649.00,25197.35,25598.00,25572.35,25425.78,1610,409.35,1030,723,44.91,-0.300324,0.824343
3,3PLAND,EQ,23-Jun-2021,13.80,13.85,14.0,13.75,13.80,13.90,13,21933,3.03,89,20431,93.15,EQ,22-Jun-2021,13.80,14.10,14.10,13.75,13.95,13.80,13.86,16898,2.34,119,13683,80.97,0.724638,1.493167
4,5PAISA,EQ,23-Jun-2021,421.70,425.70,426.6,416.00,418.00,416.75,419,19921,83.51,934,11999,60.23,EQ,22-Jun-2021,415.80,422.60,431.35,419.30,421.40,421.70,423.51,28102,119.01,2226,13090,46.58,-1.173820,0.916654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1474,ZODIACLOTH,EQ,23-Jun-2021,103.65,105.35,118.0,103.30,113.20,112.65,113,1038433,1178.47,18544,342445,32.98,EQ,22-Jun-2021,104.30,105.95,107.85,103.00,104.50,103.65,104.72,66376,69.51,1841,37671,56.75,8.683068,9.090414
1475,ZOTA,EQ,23-Jun-2021,187.30,190.90,192.5,182.20,186.50,186.25,187,185975,349.56,3438,76019,40.88,EQ,22-Jun-2021,171.25,185.45,198.50,178.00,187.05,187.30,191.21,1285003,2457.09,20257,351329,27.34,-0.560598,0.216376
1476,ZUARI,EQ,23-Jun-2021,126.95,127.90,132.0,126.20,126.55,126.75,128,184535,237.53,2720,91210,49.43,EQ,22-Jun-2021,127.85,128.90,132.40,125.00,126.20,126.95,129.32,154713,200.08,2168,78279,50.60,-0.157542,1.165191
1477,ZUARIGLOB,EQ,23-Jun-2021,115.25,117.20,119.4,115.35,118.50,117.65,118,154392,182.27,1564,112479,72.85,EQ,22-Jun-2021,116.20,118.25,118.25,114.35,116.00,115.25,116.02,79890,92.69,953,54996,68.84,2.082430,2.045221


In [12]:
merged1

,SYMBOL,SERIES_tday,DATE1_tday,PREV_CLOSE_tday,OPEN_PRICE_tday,HIGH_PRICE_tday,LOW_PRICE_tday,LAST_PRICE_tday,CLOSE_PRICE_tday,AVG_PRICE_tday,TTL_TRD_QNTY_tday,TURNOVER_LACS_tday,NO_OF_TRADES_tday,DELIV_QTY_tday,DELIV_PER_tday,SERIES_prev,DATE1_prev,PREV_CLOSE_prev,OPEN_PRICE_prev,HIGH_PRICE_prev,LOW_PRICE_prev,LAST_PRICE_prev,CLOSE_PRICE_prev,AVG_PRICE_prev,TTL_TRD_QNTY_prev,TURNOVER_LACS_prev,NO_OF_TRADES_prev,DELIV_QTY_prev,DELIV_PER_prev,%CHG,%CHG_DELIV_QTY
0,20MICRONS,EQ,23-Jun-2021,62.95,63.80,64.2,61.80,62.15,62.05,62,121773,76.40,2051,76978,63.21,EQ,14-Jun-2021,68.50,68.50,68.90,63.70,67.50,66.95,66.41,292224,194.07,3254,150619,51.54,-1.429706,0.511078
1,21STCENMGM,EQ,23-Jun-2021,20.20,20.60,20.6,20.60,20.60,20.60,20,1405,0.29,10,1405,100.00,EQ,14-Jun-2021,17.75,18.10,18.10,18.10,18.10,18.10,18.10,6411,1.16,21,6411,100.00,1.980198,0.219155
2,3MINDIA,EQ,23-Jun-2021,25572.35,25403.05,25725.5,25336.00,25464.00,25495.55,25571,1329,339.84,948,596,44.85,EQ,14-Jun-2021,26153.10,25981.00,26623.85,25799.05,25900.00,25883.35,26108.90,3573,932.87,2214,1736,48.59,-0.300324,0.343318
3,3PLAND,EQ,23-Jun-2021,13.80,13.85,14.0,13.75,13.80,13.90,13,21933,3.03,89,20431,93.15,EQ,14-Jun-2021,13.90,14.10,14.10,13.60,14.00,14.00,13.96,56241,7.85,206,47660,84.74,0.724638,0.428682
4,5PAISA,EQ,23-Jun-2021,421.70,425.70,426.6,416.00,418.00,416.75,419,19921,83.51,934,11999,60.23,EQ,14-Jun-2021,379.90,394.50,398.85,389.90,398.85,398.85,396.70,108710,431.25,2390,74147,68.21,-1.173820,0.161827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1470,ZODIACLOTH,EQ,23-Jun-2021,103.65,105.35,118.0,103.30,113.20,112.65,113,1038433,1178.47,18544,342445,32.98,EQ,14-Jun-2021,109.60,109.60,110.15,106.00,108.95,108.10,108.05,53244,57.53,1668,22135,41.57,8.683068,15.470748
1471,ZOTA,EQ,23-Jun-2021,187.30,190.90,192.5,182.20,186.50,186.25,187,185975,349.56,3438,76019,40.88,EQ,14-Jun-2021,171.40,172.00,178.50,166.20,175.00,176.20,172.90,57102,98.73,1062,21979,38.49,-0.560598,3.458711
1472,ZUARI,EQ,23-Jun-2021,126.95,127.90,132.0,126.20,126.55,126.75,128,184535,237.53,2720,91210,49.43,EQ,14-Jun-2021,124.85,123.75,134.35,117.55,130.00,130.70,129.22,1312665,1696.19,24721,440621,33.57,-0.157542,0.207003
1473,ZUARIGLOB,EQ,23-Jun-2021,115.25,117.20,119.4,115.35,118.50,117.65,118,154392,182.27,1564,112479,72.85,EQ,14-Jun-2021,119.45,122.70,122.75,114.05,118.50,119.55,119.34,314920,375.84,4536,158085,50.20,2.082430,0.711510


In [13]:
data = merged[(merged['%CHG_DELIV_QTY']>2) & (merged['%CHG']>0) & (merged[' DELIV_QTY_tday']>1000000) & (merged[' AVG_PRICE_tday']>100)]
data1 = merged1[(merged1['%CHG_DELIV_QTY']>2) & (merged1['%CHG']>0) & (merged1[' DELIV_QTY_tday']>1000000) & (merged1[' AVG_PRICE_tday']>100)]

In [14]:
data = data.sort_values(by=[' DELIV_QTY_tday','%CHG_DELIV_QTY', '%CHG'], ascending=False)
data1 = data1.sort_values(by=[' DELIV_QTY_tday','%CHG_DELIV_QTY', '%CHG'], ascending=False)

In [15]:
data = data.sort_values(by=['%CHG_DELIV_QTY', '%CHG'], ascending=False)
data1 = data1.sort_values(by=['%CHG_DELIV_QTY', '%CHG'], ascending=False)

In [16]:
data = data.reset_index(drop=True)
data1 = data1.reset_index(drop=True)

In [17]:
final = data[['SYMBOL', ' DATE1_tday',' OPEN_PRICE_tday', ' HIGH_PRICE_tday', ' LOW_PRICE_tday', ' CLOSE_PRICE_tday', ' PREV_CLOSE_tday', ' AVG_PRICE_tday',' DELIV_QTY_tday', ' DELIV_QTY_prev', ' DELIV_PER_tday', '%CHG', '%CHG_DELIV_QTY']]
final1 = data1[['SYMBOL', ' DATE1_tday',' OPEN_PRICE_tday', ' HIGH_PRICE_tday', ' LOW_PRICE_tday', ' CLOSE_PRICE_tday', ' PREV_CLOSE_tday', ' AVG_PRICE_tday',' DELIV_QTY_tday', ' DELIV_QTY_prev', ' DELIV_PER_tday', '%CHG', '%CHG_DELIV_QTY']]

In [18]:
final['Entry_Price'] = ((final[' LOW_PRICE_tday'] + final[' AVG_PRICE_tday'])/2).astype(int)
final['Target_Price'] = (final[' AVG_PRICE_tday'] + (final[' AVG_PRICE_tday']*0.05)).astype(int)

final1['Entry_Price'] = ((final1[' LOW_PRICE_tday'] + final1[' AVG_PRICE_tday'])/2).astype(int)
final1['Target_Price'] = (final1[' AVG_PRICE_tday'] + (final1[' AVG_PRICE_tday']*0.05)).astype(int)

In [19]:
# def wwma(values, n):
#     return values.ewm(alpha=1/n, adjust=False).mean()

# def atr(df, n=14):
#     data = final.copy()
#     high = data['HIGH_PRICE_tday']
#     low = data['LOW_PRICE_tday']
#     close = data['CLOSE_PRICE_tday']
#     data['tr0'] = abs(high - low)
#     data['tr1'] = abs(high - close.shift())
#     data['tr2'] = abs(low - close.shift())
#     tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
#     atr = wwma(tr, n)
#     return atr

In [20]:
final

,SYMBOL,DATE1_tday,OPEN_PRICE_tday,HIGH_PRICE_tday,LOW_PRICE_tday,CLOSE_PRICE_tday,PREV_CLOSE_tday,AVG_PRICE_tday,DELIV_QTY_tday,DELIV_QTY_prev,DELIV_PER_tday,%CHG,%CHG_DELIV_QTY,Entry_Price,Target_Price
0,CUB,23-Jun-2021,163.70,169.15,163.7,168.05,163.45,166,3585188,620508,69.65,2.814316,5.777827,164,174
1,BEL,23-Jun-2021,155.10,170.00,152.5,168.50,151.40,163,19579511,3461259,16.14,11.294584,5.656760,157,171
2,ASHOKA,23-Jun-2021,99.80,104.00,99.2,101.65,97.60,102,2179011,482791,32.92,4.149590,4.513363,100,107
3,ITI,23-Jun-2021,137.50,137.60,132.0,133.10,127.60,134,1306841,308495,17.22,4.310345,4.236182,133,140
4,OLECTRA,23-Jun-2021,193.95,212.30,191.2,212.30,193.00,204,1223386,352898,69.63,10.000000,3.466684,197,214
5,NOCIL,23-Jun-2021,215.00,225.25,215.0,217.65,214.45,220,1186047,515593,30.37,1.492189,2.300355,217,231


In [21]:
final1

,SYMBOL,DATE1_tday,OPEN_PRICE_tday,HIGH_PRICE_tday,LOW_PRICE_tday,CLOSE_PRICE_tday,PREV_CLOSE_tday,AVG_PRICE_tday,DELIV_QTY_tday,DELIV_QTY_prev,DELIV_PER_tday,%CHG,%CHG_DELIV_QTY,Entry_Price,Target_Price
0,ITI,23-Jun-2021,137.50,137.60,132.00,133.10,127.60,134,1306841,113937,17.22,4.310345,11.469856,133,140
1,MAHINDCIE,23-Jun-2021,219.00,234.80,218.00,226.80,216.85,225,1086519,110460,69.65,4.588425,9.836312,221,236
2,JKPAPER,23-Jun-2021,203.45,218.00,199.65,204.25,200.05,211,2340852,248405,11.90,2.099475,9.423530,205,221
3,BEL,23-Jun-2021,155.10,170.00,152.50,168.50,151.40,163,19579511,2378690,16.14,11.294584,8.231216,157,171
4,MARUTI,23-Jun-2021,7266.10,7479.00,7240.05,7432.70,7265.40,7417,1032732,143497,46.36,2.302695,7.196889,7328,7787
5,OLECTRA,23-Jun-2021,193.95,212.30,191.20,212.30,193.00,204,1223386,210359,69.63,10.000000,5.815706,197,214
6,INDHOTEL,23-Jun-2021,143.95,145.95,140.60,142.75,140.05,143,4341091,1282327,38.35,1.927883,3.385323,141,150
7,CUB,23-Jun-2021,163.70,169.15,163.70,168.05,163.45,166,3585188,1073157,69.65,2.814316,3.340786,164,174
8,OIL,23-Jun-2021,154.60,155.00,151.00,152.60,151.35,153,1021760,330091,43.70,0.825900,3.095389,152,160
9,ONGC,23-Jun-2021,123.50,124.40,121.75,123.35,122.05,122,9293382,3351674,42.28,1.065137,2.772758,121,128
